# The Battle of the Neighbourhoods - Hersh Oberoi 

## Introduction

In this project we are going to be analysing the city of Toronto and the neighbourhoods within it. We are going to be analysing the Boroughs of Toronto and identifying the popular cuisines when it comes to the available food options within each of the Boroughs. The problem we are trying to solve here is to identify a potential location for a new restaurant and see which particular cuisines would provide a potential opportunity for success within those locations

## Data at Use

The data set we will be using for this project comprises of the following.
We will be picking up neighbourhood information from Wikipedia - https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M.
We will be importing location, local business, popularity and other related information from the FourSquare APi.
We will also be using a variety of python libraries.

## Methodolgy

The methodolgy followed is explained alongside the coding process given below.

Import required libraries


In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json # library to handle JSON files

In [6]:
!conda install -c conda-forge geopy --yes


Solving environment: ...working... 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/win-64::anaconda==2018.12=py37_0
  - defaults/win-64::blaze==0.11.3=py37_0
  - defaults/win-64::numba==0.41.0=py37hf9181ef_0
done

## Package Plan ##

  environment location: C:\Users\Hersh\Anaconda3

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _anaconda_depends-2019.03  |           py37_0           5 KB
    anaconda-custom            |           py37_1           2 KB
    ca-certificates-2019.11.28 |       hecc5488_0         182 KB  conda-forge
    certifi-2019.11.28         |           py37_0         148 KB  conda-forge
    conda-4.8.0                |           py37_1         3.0 MB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-for

DEBUG menuinst_win32:__init__(196): Menu: name: 'Anaconda${PY_VER} ${PLATFORM}', prefix: 'C:\Users\Hersh\Anaconda3', env_name: 'None', mode: 'user', used_mode: 'user'
DEBUG menuinst_win32:create(320): Shortcut cmd is %windir%\System32\WindowsPowerShell\v1.0\powershell.exe, args are ['-ExecutionPolicy', 'ByPass', '-NoExit', '-Command', '"& \'C:\\Users\\Hersh\\Anaconda3\\shell\\condabin\\conda-hook.ps1\' ; conda activate \'C:\\Users\\Hersh\\Anaconda3\' "']


In [7]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

In [8]:
!conda install -c conda-forge folium=0.5.0 --yes

Solving environment: ...working... done

# All requested packages already installed.



In [9]:
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


## Import Toronto Neighbourhood Data

In [10]:

#Obtain Postal Code, Borough, and Neighborhood information from Wikipedia
table = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', header = 0)

#Obtain the first table
df_toronto = table[0]
df_toronto.head()

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [11]:
df_toronto.rename(columns = {"Postcode": "PostalCode", "Neighbourhood": "Neighborhood"}, inplace = True)

#Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
df_toronto.drop(df_toronto[df_toronto.Borough == 'Not assigned'].index, inplace=True)
#df.head()

#Combine the neighborhoods that exists in one postal code
df_toronto = df_toronto.groupby(['PostalCode', 'Borough'])['Neighborhood'].apply(lambda x: ','.join(x)).reset_index()
#df.head()

#Change unassigned Neighborhood to its Borough's name
df_toronto.loc[85,'Neighborhood'] = 'Queen\'s Park'

print (df_toronto.shape)

df_toronto.head()

(103, 3)


,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [12]:
#Create a dataframe of the latitude and longitudes of the Toronto Neighborhoods
latlong = pd.read_csv("http://cocl.us/Geospatial_data")
latlong.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [13]:

latlong.rename(columns = {"Postal Code": "PostalCode"}, inplace = True)
latlong.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [14]:

latlong.shape

(103, 3)

## Join neighborhood table with lat. and long. information

In [15]:

#Join the Lat and Long dataframe to Neighborhoods dataframe
df_toronto.set_index("PostalCode")
latlong.set_index("PostalCode")
neighbor=pd.merge(df_toronto, latlong)
neighbor.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [16]:
address = 'Toronto, CA'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto, Canada are {}, {}.'.format(latitude, longitude))

C:\Users\Hersh\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


The geograpical coordinate of Toronto, Canada are 43.653963, -79.387207.


## Create a map of Toronto with neighborhoods information markers

In [17]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighbor['Latitude'], neighbor['Longitude'], neighbor['Borough'], neighbor['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

## Segment and cluster only the targetted Borough in Toronto. In this case, that would be Scarborough. So, let's slice and create a new dataframe of the Scarborough Neighborhood data

In [18]:
scarborough_data = neighbor[neighbor['Borough'] == 'Scarborough'].reset_index(drop=True)
scarborough_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [19]:

scarborough_data.shape

(17, 5)

In [20]:

address = 'Scarborough, Toronto'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Scarborough, CA are {}, {}.'.format(latitude, longitude))

C:\Users\Hersh\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  after removing the cwd from sys.path.


The geograpical coordinate of Scarborough, CA are 43.773077, -79.257774.


## Create a map of Scarborough with neighborhoods information markers

In [21]:
# create map of Scarborough using latitude and longitude values
map_scarborough = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(scarborough_data['Latitude'], scarborough_data['Longitude'], scarborough_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_scarborough)  
    
map_scarborough

## Define Foursquare credentials and version

In [22]:
CLIENT_ID = 'EB4H3ZUDSUN1XKB44ANU5FTOBTEL2RTZAOU4GKAEK5D2P0WF' # your Foursquare ID
CLIENT_SECRET = 'NVI0PSOZZOTPNGVKD4MHZ4WEBJSRXJTEVKKJ0EIOHVOT1WJ4' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

## Define function to explore neighbourhoods information.

In [23]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            100)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    print('Found {} venues in {} neighborhoods.'.format(nearby_venues.shape[0], len(venues_list)))
    
    return(nearby_venues)

## Refine results by getting information specifically for our targetted borough i.e. Scarborough.

In [24]:

scarborough_venues = getNearbyVenues(names=scarborough_data['Neighborhood'],
                                   latitudes=scarborough_data['Latitude'],
                                   longitudes=scarborough_data['Longitude']
                                  )

Found 89 venues in 17 neighborhoods.


In [25]:
print(scarborough_venues.shape)
scarborough_venues.head()

(89, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge,Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,Affordable Toronto Movers,43.787919,-79.162977,Moving Target
3,"Guildwood,Morningside,West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
4,"Guildwood,Morningside,West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store


## Group the venue results by categories

In [26]:

#Venues per Neighborhood
scarborough_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,5,5,5,5,5,5
"Agincourt North,L'Amoreaux East,Milliken,Steeles East",2,2,2,2,2,2
"Birch Cliff,Cliffside West",4,4,4,4,4,4
Cedarbrae,8,8,8,8,8,8
"Clairlea,Golden Mile,Oakridge",8,8,8,8,8,8
"Clarks Corners,Sullivan,Tam O'Shanter",11,11,11,11,11,11
"Cliffcrest,Cliffside,Scarborough Village West",2,2,2,2,2,2
"Dorset Park,Scarborough Town Centre,Wexford Heights",7,7,7,7,7,7
"East Birchmount Park,Ionview,Kennedy Park",6,6,6,6,6,6


## Analyze each Neighborhood

In [27]:

print('There are {} distinct venues in {} categories.'.format(
    len(scarborough_venues['Venue'].unique()),len(scarborough_venues['Venue Category'].unique())))

#print('There are {} uniques categories.'.format(len(scarborough_venues['Venue Category'].unique())))

There are 81 distinct venues in 56 categories.


In [28]:

# one hot encoding
scarborough_onehot = pd.get_dummies(scarborough_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
scarborough_onehot['Neighborhood'] = scarborough_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [scarborough_onehot.columns[-1]] + list(scarborough_onehot.columns[:-1])
scarborough_onehot = scarborough_onehot[fixed_columns]

scarborough_onehot.head()

,Neighborhood,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Brewery,Bubble Tea Shop,Bus Line,Bus Station,Café,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,College Stadium,Convenience Store,Cosmetics Shop,Department Store,Discount Store,Electronics Store,Fast Food Restaurant,Fried Chicken Joint,Gaming Cafe,Gas Station,General Entertainment,Grocery Store,Hakka Restaurant,Hobby Shop,Indian Restaurant,Intersection,Italian Restaurant,Korean Restaurant,Latin American Restaurant,Lounge,Medical Center,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Motel,Moving Target,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Rental Car Location,Sandwich Place,Shopping Mall,Skating Rink,Soccer Field,Supermarket,Thai Restaurant,Vietnamese Restaurant
0,"Rouge,Malvern",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Highland Creek,Rouge Hill,Port Union",0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Highland Creek,Rouge Hill,Port Union",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Guildwood,Morningside,West Hill",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
4,"Guildwood,Morningside,West Hill",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


## Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [29]:
scarborough_grouped = scarborough_onehot.groupby('Neighborhood').mean().reset_index()
scarborough_grouped

,Neighborhood,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Brewery,Bubble Tea Shop,Bus Line,Bus Station,Café,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,College Stadium,Convenience Store,Cosmetics Shop,Department Store,Discount Store,Electronics Store,Fast Food Restaurant,Fried Chicken Joint,Gaming Cafe,Gas Station,General Entertainment,Grocery Store,Hakka Restaurant,Hobby Shop,Indian Restaurant,Intersection,Italian Restaurant,Korean Restaurant,Latin American Restaurant,Lounge,Medical Center,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Motel,Moving Target,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Rental Car Location,Sandwich Place,Shopping Mall,Skating Rink,Soccer Field,Supermarket,Thai Restaurant,Vietnamese Restaurant
0,Agincourt,0.0,0.000,0.000000,0.000000,0.000000,0.0,0.200000,0.000000,0.000000,0.000,0.000,0.00,0.000,0.000000,0.2,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.00,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.00,0.2,0.2,0.000000,0.000,0.000000,0.000000,0.0,0.0,0.000000,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.20,0.000,0.000000,0.000000,0.000000
1,"Agincourt North,L'Amoreaux East,Milliken,Steel...",0.0,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000,0.000,0.00,0.000,0.000000,0.0,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.00,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.0,0.000000,0.000,0.000000,0.000000,0.0,0.0,0.000000,0.500,0.000000,0.000000,0.000000,0.5,0.000000,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.000000
2,"Birch Cliff,Cliffside West",0.0,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000,0.000,0.25,0.000,0.000000,0.0,0.000000,0.25,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.25,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.0,0.000000,0.000,0.000000,0.000000,0.0,0.0,0.000000,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.25,0.000,0.000000,0.000000,0.000000
3,Cedarbrae,0.0,0.125,0.000000,0.125000,0.125000,0.0,0.000000,0.000000,0.000000,0.000,0.000,0.00,0.125,0.000000,0.0,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.125000,0.000000,0.125,0.00,0.000000,0.125,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.0,0.000000,0.000,0.000000,0.000000,0.0,0.0,0.000000,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.000,0.000000,0.125000,0.000000
4,"Clairlea,Golden Mile,Oakridge",0.0,0.000,0.000000,0.250000,0.000000,0.0,0.000000,0.000000,0.000000,0.125,0.125,0.00,0.000,0.000000,0.0,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.125000,0.000000,0.000000,0.000,0.00,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.0,0.000000,0.125,0.000000,0.000000,0.0,0.0,0.000000,0.125,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.125,0.000000,0.000000,0.000000
5,"Clarks Corners,Sullivan,Tam O'Shanter",0.0,0.000,0.000000,0.000000,0.090909,0.0,0.000000,0.000000,0.000000,0.000,0.000,0.00,0.000,0.090909,0.0,0.000000,0.00,0.090909,0.0,0.000000,0.000000,0.000000,0.090909,0.090909,0.000000,0.000,0.00,0.000000,0.000,0.000000,0.000000,0.000000,0.090909,0.00,0.0,0.0,0.000000,0.000,0.000000,0.000000,0.0,0.0,0.090909,0.000,0.000000,0.090909,0.181818,0.0,0.000000,0.000000,0.000000,0.00,0.000,0.000000,0.090909,0.000000
6,"Cliffcrest,Cliffside,Scarborough Village West",0.5,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000,0.000,0.00,0.000,0.000000,0.0,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.00,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.0,0.000000,0.000,0.000000,0.000000,0.5,0.0,0.000000,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.000000
7,"Dorset Park,Scarborough Town Centre,Wexford He...",0.0,0.

## Each neighborhood with the top 5 venues

In [30]:
num_top_venues = 10

for hood in scarborough_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = scarborough_grouped[scarborough_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0             Clothing Store   0.2
1               Skating Rink   0.2
2             Breakfast Spot   0.2
3  Latin American Restaurant   0.2
4                     Lounge   0.2
5               Noodle House   0.0
6               Intersection   0.0
7         Italian Restaurant   0.0
8          Korean Restaurant   0.0
9             Medical Center   0.0


----Agincourt North,L'Amoreaux East,Milliken,Steeles East----
                       venue  freq
0                 Playground   0.5
1                       Park   0.5
2        American Restaurant   0.0
3              Moving Target   0.0
4          Indian Restaurant   0.0
5               Intersection   0.0
6         Italian Restaurant   0.0
7          Korean Restaurant   0.0
8  Latin American Restaurant   0.0
9                     Lounge   0.0


----Birch Cliff,Cliffside West----
                       venue  freq
0            College Stadium  0.25
1      General Entertainment  0.25
2     

## Put the above information into a pandas dataframe

In [31]:

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

## Create a new dataframe and display the top ten venues for each neighborhood

In [32]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = scarborough_grouped['Neighborhood']

for ind in np.arange(scarborough_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(scarborough_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Clothing Store,Skating Rink,Breakfast Spot,Latin American Restaurant,Lounge,Vietnamese Restaurant,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
1,"Agincourt North,L'Amoreaux East,Milliken,Steel...",Playground,Park,Vietnamese Restaurant,Chinese Restaurant,Gaming Cafe,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store
2,"Birch Cliff,Cliffside West",Skating Rink,College Stadium,Café,General Entertainment,Vietnamese Restaurant,Clothing Store,Gaming Cafe,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
3,Cedarbrae,Thai Restaurant,Athletics & Sports,Hakka Restaurant,Bakery,Bank,Gas Station,Fried Chicken Joint,Caribbean Restaurant,College Stadium,Gaming Cafe
4,"Clairlea,Golden Mile,Oakridge",Bakery,Metro Station,Soccer Field,Fast Food Restaurant,Bus Station,Park,Bus Line,Convenience Store,Coffee Shop,College Stadium
5,"Clarks Corners,Sullivan,Tam O'Shanter",Pizza Place,Fried Chicken Joint,Chinese Restaurant,Noodle House,Convenience Store,Pharmacy,Italian Restaurant,Bank,Fast Food Restaurant,Thai Restaurant
6,"Cliffcrest,Cliffside,Scarborough Village West",American Restaurant,Motel,General Entertainment,Gaming Cafe,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Cosmetics Shop
7,"Dorset Park,Scarborough Town Centre,Wexford He...",Indian Restaurant,Vietnamese Restaurant,Gaming Cafe,Brewery,Pet Store,Chinese Restaurant,Clothing Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
8,"East Birchmount Park,Ionview,Kennedy Park",Discount Store,Hobby Shop,Department Store,Convenience Store,Coffee Shop,Vietnamese Restaurant,General Entertainment,Gaming Cafe,Fried Chicken Joint,Fast Food Restaurant
9,"Guildwood,Morningside,West Hill",Mexican Restaurant,Electronics Store,Intersection,Rental Car Location,Breakfast Spot,Pizza Place,Medical Center,Vietnamese Restaurant,Fast Food Restaurant,Discount Store


In [33]:

neighborhoods_venues_sorted.iloc[11,]

Neighborhood                   L'Amoreaux West
1st Most Common Venue            Grocery Store
2nd Most Common Venue       Chinese Restaurant
3rd Most Common Venue     Fast Food Restaurant
4th Most Common Venue              Coffee Shop
5th Most Common Venue          Bubble Tea Shop
6th Most Common Venue           Breakfast Spot
7th Most Common Venue                 Pharmacy
8th Most Common Venue              Pizza Place
9th Most Common Venue           Sandwich Place
10th Most Common Venue             Supermarket
Name: 11, dtype: object

## Cluster the Scarborough Neighborhood using k-means
Run K-means to cluster neighborhood into three clusters

In [34]:

# set number of clusters
kclusters = 3

scarborough_grouped_clustering = scarborough_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=2).fit(scarborough_grouped_clustering)

# check cluster labels generated for each row in the dataframe
#kmeans.labels_[0:10] 
kmeans.labels_

array([1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 0, 1])

## Create a new dataframe that includes the cluster as well as the top ten venues for each neighborhood

In [35]:

#Note that the neighborhood Upper Rouge does not have any venues, so I will drop from dataset
scarborough_data.drop(scarborough_data[scarborough_data.Neighborhood == 'Upper Rouge'].index, inplace = True)
#df_toronto.drop(df_toronto[df_toronto.Borough == 'Not assigned'].index, inplace=True)

scarborough_merged = scarborough_data

# add clustering labels
scarborough_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
scarborough_merged = scarborough_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

scarborough_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353,1,Fast Food Restaurant,Vietnamese Restaurant,Thai Restaurant,Gas Station,Gaming Cafe,Fried Chicken Joint,Electronics Store,Discount Store,Department Store,Cosmetics Shop
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,0,Moving Target,Bar,Chinese Restaurant,Gaming Cafe,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Cosmetics Shop
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711,1,Mexican Restaurant,Electronics Store,Intersection,Rental Car Location,Breakfast Spot,Pizza Place,Medical Center,Vietnamese Restaurant,Fast Food Restaurant,Discount Store
3,M1G,Scarborough,Woburn,43.770992,-79.216917,1,Coffee Shop,Indian Restaurant,Korean Restaurant,Vietnamese Restaurant,Clothing Store,Gaming Cafe,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,1,Thai Restaurant,Athletics & Sports,Hakka Restaurant,Bakery,Bank,Gas Station,Fried Chicken Joint,Caribbean Restaurant,College Stadium,Gaming Cafe


## Create a map using the above dataframe

In [36]:

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(scarborough_merged['Latitude'], scarborough_merged['Longitude'], scarborough_merged['Neighborhood'], scarborough_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine Scarborough Neighbourhood Clusters
Examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, assign a name to each cluster
## Scarborough Clusters 0, 1, 2

In [37]:

scarborough_cluster_0 = scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 0, scarborough_merged.columns[[1] + list(range(4, scarborough_merged.shape[1]))]]

scarborough_cluster_1 = scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 1, scarborough_merged.columns[[1] + list(range(4, scarborough_merged.shape[1]))]]

scarborough_cluster_2 = scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 2, scarborough_merged.columns[[1] + list(range(4, scarborough_merged.shape[1]))]]

In [38]:

scarborough_cluster_0

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Scarborough,-79.160497,0,Moving Target,Bar,Chinese Restaurant,Gaming Cafe,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store,Cosmetics Shop
14,Scarborough,-79.284577,0,Playground,Park,Vietnamese Restaurant,Chinese Restaurant,Gaming Cafe,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store


scarborough_cluster_1

In [39]:
scarborough_cluster_2

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,Scarborough,-79.304302,2,Pizza Place,Fried Chicken Joint,Chinese Restaurant,Noodle House,Convenience Store,Pharmacy,Italian Restaurant,Bank,Fast Food Restaurant,Thai Restaurant


## Results

Based on the above information, we have identified the above conclusions

    Cluster 2 in Scarborough is clearly the largest with the largest audience size and hence, opportunity. 
    
    There is not neccessarily a singular cuisine or theme that can be identified to have an extremely strong or dominating         presence. 
    
    A fairly significant presence in top 10 popular venues for Chinese Restaurants, Indian Restaurants and Vietnamese         Restaurants 

## Conclusions

Based on the above results, we simply do not have enough conclusive data to identify an extremely dominant market space when it comes to the food industry. There are clearly strong players with a somewhat larger presence i.e. Chinese, Indian and Vietnamese restaurants. This can be concluded to provide one of two paths of decision making. Either the market can be considered saturated with these opportunities or it can be considered to be a safe option to invest in considering the large demand.
Overall though, our study has been somewhat inconclusive from a business opportunity research point of view.